In [1]:
import numpy as np 
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from collections import Counter

In [2]:
ds = pd.read_csv('api_call_frequency.csv')

RegQueryValueExA    0.062827
47                   FindFirstFileExW    0.049722
100                   RegCreateKeyExW    0.049544
17                      RegOpenKeyExA    0.049187
76                 SetFileAttributesW    0.049180
253                CreateRemoteThread    0.037693
6                       NtOpenSection    0.032787
103                    RegSetValueExW    0.030945
32                   CreateDirectoryW    0.027724
131                GetVolumePathNameW    0.026586
197                CryptProtectMemory    0.024580
59                        GetFileType    0.024357
43                NtFreeVirtualMemory    0.023789
94                         setsockopt    0.020172
124                  RegQueryInfoKeyW    0.019329
44                         NtOpenFile    0.019133
150        SHGetSpecialFolderLocation    0.016623
55                    FindResourceExW    0.016407
38   GetVolumePathNamesForVolumeNameW    0.016393
75              MoveFileWithProgressW    0.016120

In [3]:
df = ds[['RegQueryValueExA', 'FindFirstFileExW', 'RegCreateKeyExW', 'RegOpenKeyExA', 'SetFileAttributesW', 'CreateRemoteThread', 'NtOpenSection','label']]

In [4]:
df.shape

(3083, 8)

In [6]:
df.sample(10)

,RegQueryValueExA,FindFirstFileExW,RegCreateKeyExW,RegOpenKeyExA,SetFileAttributesW,CreateRemoteThread,NtOpenSection,label
890,1,0,0,6,0,0,0,1
1399,1,88,0,7,12,0,10,0
2244,0,5,0,0,0,0,0,1
1175,0,0,0,0,0,0,0,1
1674,0,0,0,2,0,0,0,1
1569,0,0,1,0,0,0,0,1
2539,0,0,0,0,0,0,0,1
2320,0,0,0,0,0,0,0,1
2701,0,0,0,0,0,0,0,1
3074,0,16,0,0,0,0,0,1


In [8]:
## Start Tracing 
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

X_train, X_test, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
print(ytest.shape)

(617,)


In [17]:
X_test.sample(10)

,RegQueryValueExA,FindFirstFileExW,RegCreateKeyExW,RegOpenKeyExA,SetFileAttributesW,CreateRemoteThread,NtOpenSection
30,10,921,10,69,189,0,17
368,1,60,0,7,9,0,10
1075,171,18,4,345,0,0,1
1226,0,0,0,0,0,0,0
1412,0,0,0,0,0,0,0
2110,0,5,0,0,0,0,0
1987,0,0,0,0,0,0,0
965,0,0,0,0,0,0,0
2616,0,0,0,0,0,0,0
838,0,0,0,0,0,0,0


In [12]:
RF = RandomForestClassifier(n_estimators=9, oob_score=True, random_state=42)
RF.fit(X_train, ytrain)

c:\USERS\NEEZY\ANACONDA3\lib\site-packages\sklearn\ensemble\_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestClassifier(n_estimators=9, oob_score=True, random_state=42)

In [13]:
ypred = RF.predict(X_test)
acc = accuracy_score(ytest, ypred)
print(f"Accuracy of model: {acc:4f}")
cm = confusion_matrix(ytest, ypred) 

Accuracy of model: 0.975689


In [14]:
print(RF.oob_score_)

0.9545823195458232


In [15]:
test_sample = [[1, 60, 0, 7, 9, 0, 10]]
test_sample_2 = [[171, 18, 4, 345, 0, 0, 1]]
modified_sample = [[171,208,4,345,0,0,43]]

In [45]:

predict_sample=RF.predict() 
print(f"This sample is:{predict_sample[0]}")

This sample is:0


c:\USERS\NEEZY\ANACONDA3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [37]:
modified_sample = [[171,208,4,345,0,0,43]]
race = np.array(modified_sample)  

In [38]:
tree_predictions = [tree.predict(race)[0] for tree in RF.estimators_]

In [39]:
print(tree_predictions)

[0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0]


In [40]:
from collections import Counter

vote_counts = Counter(tree_predictions)
print(vote_counts)

Counter({0.0: 5, 1.0: 4})


In [16]:
from sklearn.tree import export_graphviz
RF2 = RandomForestClassifier(n_estimators=100, random_state=42)
RF2.fit(X, y)  

estimator = RF2.estimators_[0]


export_graphviz(
    estimator,
    out_file='tree_5.dot',
    feature_names=X.columns,  
    class_names=[str(cls) for cls in y.unique()],  
    rounded=True,
    proportion=False,
    precision=2,
    filled=True
)
